# Pipeline Project

You will be using the provided data to create a machine learning model pipeline.

You must handle the data appropriately in your pipeline to predict whether an
item is recommended by a customer based on their review.
Note the data includes numerical, categorical, and text data.

You should ensure you properly train and evaluate your model.

## The Data

The dataset has been anonymized and cleaned of missing values.

There are 8 features for to use to predict whether a customer recommends or does
not recommend a product.
The `Recommended IND` column gives whether a customer recommends the product
where `1` is recommended and a `0` is not recommended.
This is your model's target/

The features can be summarized as the following:

- **Clothing ID**: Integer Categorical variable that refers to the specific piece being reviewed.
- **Age**: Positive Integer variable of the reviewers age.
- **Title**: String variable for the title of the review.
- **Review Text**: String variable for the review body.
- **Positive Feedback Count**: Positive Integer documenting the number of other customers who found this review positive.
- **Division Name**: Categorical name of the product high level division.
- **Department Name**: Categorical name of the product department name.
- **Class Name**: Categorical name of the product class name.

The target:
- **Recommended IND**: Binary variable stating where the customer recommends the product where 1 is recommended, 0 is not recommended.

## Load Data

In [2]:
import pandas as pd

# Load data
df = pd.read_csv(
    'data/reviews.csv',
)

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18442 entries, 0 to 18441
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Clothing ID              18442 non-null  int64 
 1   Age                      18442 non-null  int64 
 2   Title                    18442 non-null  object
 3   Review Text              18442 non-null  object
 4   Positive Feedback Count  18442 non-null  int64 
 5   Division Name            18442 non-null  object
 6   Department Name          18442 non-null  object
 7   Class Name               18442 non-null  object
 8   Recommended IND          18442 non-null  int64 
dtypes: int64(4), object(5)
memory usage: 1.3+ MB


,Clothing ID,Age,Title,Review Text,Positive Feedback Count,Division Name,Department Name,Class Name,Recommended IND
0,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,0,General,Dresses,Dresses,0
1,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",0,General Petite,Bottoms,Pants,1
2,847,47,Flattering shirt,This shirt is very flattering to all due to th...,6,General,Tops,Blouses,1
3,1080,49,Not for the very petite,"I love tracy reese dresses, but this one is no...",4,General,Dresses,Dresses,0
4,858,39,Cagrcoal shimmer fun,I aded this in my basket at hte last mintue to...,1,General Petite,Tops,Knits,1


## Preparing features (`X`) & target (`y`)

In [3]:
data = df

# separate features from labels
X = data.drop('Recommended IND', axis=1)
y = data['Recommended IND'].copy()

print('Labels:', y.unique())
print('Features:')
display(X.head())

Labels: [0 1]
Features:


,Clothing ID,Age,Title,Review Text,Positive Feedback Count,Division Name,Department Name,Class Name
0,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,0,General,Dresses,Dresses
1,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",0,General Petite,Bottoms,Pants
2,847,47,Flattering shirt,This shirt is very flattering to all due to th...,6,General,Tops,Blouses
3,1080,49,Not for the very petite,"I love tracy reese dresses, but this one is no...",4,General,Dresses,Dresses
4,858,39,Cagrcoal shimmer fun,I aded this in my basket at hte last mintue to...,1,General Petite,Tops,Knits


In [ ]:
# Split data into train and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.4, #changed the test size to prevent overfitting, splitting into train,test,val would be a better option
    shuffle=True,
    random_state=27,
)

# Your Work

## Data Exploration

In [5]:
X.describe()

,Clothing ID,Age,Positive Feedback Count
count,18442.000000,18442.000000,18442.000000
mean,954.896757,43.383635,2.697484
std,141.571783,12.246264,5.942220
min,2.000000,18.000000,0.000000
25%,863.000000,34.000000,0.000000
50%,952.000000,41.000000,1.000000
75%,1078.000000,52.000000,3.000000
max,1205.000000,99.000000,122.000000


## EXTRA FEATURE: Custom spacy textcat

code is commented out as it does not need to run again

In [ ]:
#!python -m spacy download en_core_web_lg
from pathlib import Path
from spacy.cli.init_config import init_config
#init_config(lang='en',pipeline=['textcat'],optimize="accuracy", gpu=True).to_disk(Path("base_config.cfg"))

#the config was changed to freeze the tok2vec

In [ ]:
from spacy.tokens import DocBin
import spacy


def to_spacy_file(X, y, file):
    '''
    function converts data into spacy format and saves it to disk

    inputs: x: dataframe, features
            y: series, label
            file: Path

    outputs: none
    '''
    
    docs = DocBin()
    nlp = spacy.blank('en')
    for i, doc in enumerate(nlp.pipe(X['Title'] + '\n' + X['Review Text'])):
        doc.cats = {'Reccommended': y.values[i],
                    'NOT_Reccommended': 1-y.values[i]
                    }
        docs.add(doc)
    docs.to_disk(file)

#to_spacy_file(X_train,y_train,Path("train.spacy"))
#to_spacy_file(X_test,y_test,Path("test.spacy"))

In [ ]:
#train the spacy model described in config using the gpu
#pip could build the wheel for blis when installing spacy[cuda12x] so i switched to conda
from spacy.cli.train import train
#spacy.require_gpu()
#train(Path("config.cfg"), overrides={"paths.train": str(Path("train.spacy").resolve()), "paths.dev": str(Path("test.spacy").resolve())},use_gpu=0,output_path=Path('model/'))

ℹ Saving to output directory: model
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'textcat']
ℹ Frozen components: ['tok2vec']
ℹ Set annotations on update for: ['tok2vec']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTCAT  CATS_SCORE  SCORE 
---  ------  ------------  ----------  ------
  0       0          0.25       45.17    0.45
  0     200         29.02       45.33    0.45
  0     400         26.17       65.17    0.65
  0     600         34.22       45.35    0.45
  0     800         33.62       59.67    0.60
  0    1000         30.68       68.95    0.69
  0    1200         26.56       74.42    0.74
  0    1400         24.42       78.35    0.78
  0    1600         20.84       78.63    0.79
  0    1800         17.83       63.31    0.63
  0    2000         17.81       74.27    0.74
  0    2200         18.23    

## Building Pipeline

In [55]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

class SpacyClassifier(BaseEstimator, TransformerMixin):

    nlp = spacy.load('model/model-last/')

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return np.array([
            [doc.cats['Reccommended']]
            for doc in self.nlp.pipe(X['Title'] + '\n' + X['Review Text'])
        ])

<h3>Disclaimer</h3>
In the following section I reuse a lot of code from the previous assignment 
Data Scientist Nanodegree Course 4: Data science Pipelines / NLP Pipelines / Exercise: NLP Pipelines

this is because the assignments are very similar and the nature of pipelines is very plug and play resulting in similar code anyway

In [56]:
# analogous to Exercise: NLP Pipelines -- https://www.udacity.com/

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

num_features = ['Positive Feedback Count', 'Clothing ID', 'Age']
cat_features = ['Division Name', 'Department Name', 'Class Name']
text_features = ['Title', 'Review Text']

num_pipeline = Pipeline([
    (
        'scaler',
        MinMaxScaler(),
    )
])

cat_pipeline = Pipeline([
    (
        'cat_encoder',
        OneHotEncoder(
            sparse_output=False,
            handle_unknown='ignore',
        )
    )
])

feature_pipe = ColumnTransformer([
        ('num', num_pipeline, num_features),
        ('cat', cat_pipeline, cat_features),
        ('text', SpacyClassifier(), text_features)
])

## Training Pipeline

In [57]:
# analogous to Exercise: NLP Pipelines -- https://www.udacity.com/
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline

model_pipeline = make_pipeline(
    feature_pipe,
    RandomForestClassifier(random_state=42),
)

model_pipeline.fit(X_train, y_train)

,steps,"[('columntransformer', ...), ('randomforestclassifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...), ...]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [58]:
# credit: Exercise: NLP Pipelines -- https://www.udacity.com/
from sklearn.metrics import accuracy_score

y_pred_forest_pipeline = model_pipeline.predict(X_test)
accuracy_forest_pipeline = accuracy_score(y_test, y_pred_forest_pipeline)

print('Accuracy:', accuracy_forest_pipeline)

Accuracy: 0.9019926799511997


## Fine-Tuning Pipeline

In [ ]:
# the SpacyClassifier causes serius data leakage as it is not part of the training pipeline.
# doing cross validation and hyperparameter tuning on the SpacyClassifier is too resource intensive and out of scope
# when building such a pipeline, the spacy classifier could recieve losses from the random forest to increase accuracy further


# analogous to Exercise: NLP Pipelines -- https://www.udacity.com/
from sklearn.model_selection import GridSearchCV

my_distributions = dict(
    randomforestclassifier__max_features=[
        'sqrt',
        'log2'
    ],
    randomforestclassifier__n_estimators=[
        150,
        200,
    ],
)

param_search = GridSearchCV(
    estimator=model_pipeline,
    param_grid=my_distributions,
    cv=4,         # Use 4-fold cross-validation
    n_jobs=1,    # usignd multi ple processes results in pickling errors
    refit=True,   # Refit the model using the best parameters found
    verbose=3,    # Output of parameters, score, time
)

param_search.fit(X_train, y_train)

# Retrieve the best parameters
param_search.best_params_

Fitting 4 folds for each of 4 candidates, totalling 16 fits
[CV 1/4] END randomforestclassifier__max_features=sqrt, randomforestclassifier__n_estimators=150;, score=0.978 total time=   6.1s
[CV 2/4] END randomforestclassifier__max_features=sqrt, randomforestclassifier__n_estimators=150;, score=0.975 total time=   6.0s
[CV 3/4] END randomforestclassifier__max_features=sqrt, randomforestclassifier__n_estimators=150;, score=0.984 total time=   5.9s
[CV 4/4] END randomforestclassifier__max_features=sqrt, randomforestclassifier__n_estimators=150;, score=0.981 total time=   6.1s
[CV 1/4] END randomforestclassifier__max_features=sqrt, randomforestclassifier__n_estimators=200;, score=0.977 total time=   6.2s
[CV 2/4] END randomforestclassifier__max_features=sqrt, randomforestclassifier__n_estimators=200;, score=0.975 total time=   6.1s
[CV 3/4] END randomforestclassifier__max_features=sqrt, randomforestclassifier__n_estimators=200;, score=0.984 total time=   6.3s
[CV 4/4] END randomforestclass

{'randomforestclassifier__max_features': 'log2',
 'randomforestclassifier__n_estimators': 200}

In [ ]:
# credit: Exercise: NLP Pipelines -- https://www.udacity.com/
model_best = param_search.best_estimator_
y_pred_forest_pipeline = model_best.predict(X_test)
accuracy_forest_pipeline = accuracy_score(y_test, y_pred_forest_pipeline)

print('Accuracy:', accuracy_forest_pipeline)

Accuracy: 0.9021282364104649
